In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/diabetescsv/diabetes.csv


In [4]:
dataset= pd.read_csv('/kaggle/input/diabetescsv/diabetes.csv')

In [5]:
dataset.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

In [9]:
len(dataset)

768

Now we will analyze that whether this dataset have any null value in unexpected column and if there is any then we will have to handle that 
first of all check that whether there is any missing data in the dataset or not

In [10]:
dataset.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

Clearly there is not any missing data in any column.
Now we will check for null value in all the columns one by one 

In [14]:
print('BP:',dataset[dataset.BloodPressure==0].shape[0])
print('Glucose:',dataset[dataset.Glucose==0].shape[0])
print('Skin thickness:',dataset[dataset.SkinThickness==0].shape[0])
print('Insulin:',dataset[dataset.Insulin==0].shape[0])
print('BMI:',dataset[dataset.BMI==0].shape[0])

BP: 35
Glucose: 5
Skin thickness: 227
Insulin: 374
BMI: 11


This means there are 35 values that are null in BP column, 5 values that are null in glucose column, 227 values that are null in Skin Thickness column,  374 values that are null in Insulin column,11 values that are null in BMI column column but it is not possible in real life to have zero values in these sections thus we will have to handle it.

We will replace them with mean value.

In [15]:
columns_replace= ['BloodPressure', 'Glucose', 'SkinThickness', 'Insulin', 'BMI']

We need to replace 0 with NaN so that we can aply mean function here, also skipna=true will be set to ignore NaN entries and calculate the mean without counting NaN entries.

In [17]:

for column in columns_replace:
    dataset[column] = dataset[column].replace(0,np.NaN)
    mean= int(dataset[column].mean(skipna=True))
    dataset[column] = dataset[column].replace(np.NaN,mean)
    
    

check that we are not having any null value now

In [18]:
print('BP:',dataset[dataset.BloodPressure==0].shape[0])
print('Glucose:',dataset[dataset.Glucose==0].shape[0])
print('Skin thickness:',dataset[dataset.SkinThickness==0].shape[0])
print('Insulin:',dataset[dataset.Insulin==0].shape[0])
print('BMI:',dataset[dataset.BMI==0].shape[0])

BP: 0
Glucose: 0
Skin thickness: 0
Insulin: 0
BMI: 0


Split the dataset into training and testing data.

result(y) is the outcome row and training and testing will be done on rest other rows

In [21]:
x=dataset.iloc[:, 0:8]
y=dataset.iloc[:, 8]

x_train, x_test, y_train,y_test= train_test_split(x,y, random_state=30, test_size=.2)

[reference for Standardization of data](https://towardsdatascience.com/how-and-why-to-standardize-your-data-996926c2c832)

In [22]:
sc= StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

for selecting value of k

In [77]:
#but this is creating a little low accuracy model so i look around with hit and trial 
#import math
#math.sqrt(len(y_train))

In [78]:
model=KNeighborsClassifier(n_neighbors = 47, p=2, metric= 'euclidean' )

model.fit(x_train, y_train)


KNeighborsClassifier(metric='euclidean', n_neighbors=47)

In [79]:
y_pred= model.predict(x_test)
print(y_pred)

[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0
 1 1 0 0 1 0 0 1 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 1 1 1 0 0 1 0 0
 0 1 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 1 1 1 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 1 0 1 0 0 0 1 0 0 0 0 0 1 0
 0 1 0 0 1 0]


In [80]:
#evaluate matrix
c= confusion_matrix(y_test, y_pred)
# this will give matrix of prediction and actual value 
print(c)

[[94 10]
 [22 28]]


In [81]:
print('F1_score:', f1_score(y_test, y_pred))
print('accuracy_score:', accuracy_score(y_test, y_pred))

F1_score: 0.6363636363636364
accuracy_score: 0.7922077922077922
